# **💼 Capstone Project: Predicting Developer Salaries Using Stack Overflow Survey Data (2023–2024)**

---

## **🧠 Business Understanding**

### 🎯 Project Objective
Build a machine learning model that predicts annual developer salaries using demographic, professional, and technical features. We leverage Stack Overflow’s 2023 and 2024 survey datasets to inform both global and localized (Kenyan) compensation insights.

### ❓ Why This Matters
Salary transparency is limited in many global regions — especially across Africa. Developers often navigate job transitions, promotions, or freelance pricing without reliable benchmarks. This model aims to provide data-driven insights that help both job seekers and hiring managers make informed compensation decisions.

This model helps:
- Developers **benchmark expected compensation**
- Employers **set fair, competitive pay**
- Career changers **evaluate the ROI of learning paths**
- HR platforms **integrate salary prediction engines**
- Policymakers and analysts **understand wage trends in tech**

### 🌍 Kenya-Specific Relevance
In Kenya, tech hubs like Nairobi are booming — yet salary data remains fragmented. This model could:
- Help junior/mid-level developers negotiate better
- Empower remote-first hiring with global salary range visibility
- Be integrated into job platforms like **Fuzu**, **BrighterMonday**, or **Andela**

### 🧑‍💻 Industry Domains
- **Primary**: Technology
- **Secondary**: Human Capital Analytics, Labor Market Research

### 👥 Stakeholders

| Stakeholder Group         | Value Proposition                                                                 |
|---------------------------|------------------------------------------------------------------------------------|
| Developers                | Benchmark realistic compensation based on skills and experience                   |
| Employers & Recruiters    | Offer data-driven, competitive salaries                                            |
| HR Tech Platforms         | Integrate model into job boards or career guidance tools                          |
| Bootcamps & Career Coaches| Showcase expected returns on upskilling efforts                                   |
| Policy & Advocacy Groups  | Inform labor market planning and economic inclusion initiatives                   |

### 🔬 Literature & Prior Work
This project builds on:
- Previous Stack Overflow Salary Calculators (now deprecated)
- ML-based salary models using Random Forest, XGBoost, etc.
- Global developer reports (GitHub Octoverse, Dev.to, HackerRank)

### ✨ Unique Contribution
- Combines recent **multi-year (2023 & 2024)** datasets
- Applies **localized lens** for Kenya/Africa
- Prioritizes **real-world use cases** for talent, HR, and learning ecosystems

### 🎯 Success Metrics
- Overall RMSE: < 0.45 (log scale)
- SSA/Kenya RMSE: < 0.45
- R² Score: > 0.4
- Validation: Use 20% holdout set and 5-fold stratified CV, validated against Kenya salary ranges from local job boards.

### ✅ Validation Approach
- Reserve 20% of data as a holdout set for final evaluation.
- Use stratified 5-fold CV to ensure SSA/Kenya representation.
- Cross-check predictions with Kenya salary data from Fuzu or Andela.

---

## **📊 Data Understanding**

### 📁 Data Sources
- `survey_results_public_2023.csv`
- `survey_results_public_2024.csv`

Source: [Stack Overflow Developer Survey](https://insights.stackoverflow.com/survey)

### 📐 Data Summary

| Year | Total Responses | Countries | SSA Responses | Kenya Responses |
|------|------------------|-----------|----------------|------------------|
| 2023 | ~89,000          | ~180      | 1,828          | 244              |
| 2024 | ~65,000          | ~188      | 1,271          | 180              |

Combined, we expect **100,000+ usable rows** across ~188 countries.

### 🔑 Key Features

| Category        | Variables (examples)                                                  |
|----------------|------------------------------------------------------------------------|
| Demographics    | `Age`, `Country`, `Gender`                                             |
| Education       | `EdLevel`, `LearnCode`, `YearsCodePro`                                 |
| Employment      | `Employment`, `OrgSize`, `RemoteWork`                                  |
| Technical Tools | `LanguageHaveWorkedWith`, `PlatformHaveWorkedWith`, `AISelect`         |
| Target Variable | `ConvertedCompYearly` (Annual salary in USD)                           |
| Meta Info       | `SurveyYear`, `Currency`                                               |

### ⚠️ Data Limitations & Mitigation Strategies

| Limitation                             | Proposed Mitigation                                           |
|----------------------------------------|---------------------------------------------------------------|
| Schema changes between years           | Focus on **common columns**; map others as needed             |
| Imbalanced country distribution        | Consider **oversampling**, stratified models                  |
| Salary skew and outliers               | Apply **log transformation**, drop outliers via IQR/z-score   |
| Sparse Kenya data                      | Enrich with African subset or train global model              |
| Missing values                         | Drop, impute, or bin categories during preprocessing          |

### 💾 Planned Enhancements
- Add `SurveyYear` column to track year-based differences
- Combine datasets using common schema

---

In [ ]:
# Import relevant libraries
# warnings
import warnings
warnings.filterwarnings('ignore')
# data handling
import pandas as pd 
import numpy as np
# visualisation
import matplotlib.pyplot as plt
import seaborn as sns
# preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
# models
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
# evaluation
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# explainability
import shap
# utilities
import os
from pathlib import Path
# feature selection and importance
from sklearn.feature_selection import SelectKBest, f_regression

### 1. Load 2023 & 2024 Stack Overflow Survey Data

In [ ]:
# loading the 2023 and 2024 datasets
df_2023 = pd.read_csv('survey_results_public_2023.csv')
df_2024 = pd.read_csv('survey_results_public_2024.csv')


### 2. Understand Schema Differences Between the Two Years

In [ ]:

print(df_2023.columns.difference(df_2024.columns))
print(df_2024.columns.difference(df_2023.columns))

Stack Overflow's survey questions vary slightly each year. We checked for columns that appear in one year but not the other. This helps us align the two datasets before combining them.

In [ ]:
df_2023.info()

In [ ]:
df_2024.info()

### 3. Merge Datasets for a Unified View

In [ ]:
# Add SurveyYear column to distinguish entries
df_2023['SurveyYear'] = 2023
df_2024['SurveyYear'] = 2024

# Align column structure by filling in missing columns in each dataset
missing_2023 = df_2024.columns.difference(df_2023.columns)
missing_2024 = df_2023.columns.difference(df_2024.columns)

# Add missing columns to each dataframe and fill with NaN
for col in missing_2023:
    df_2023[col] = np.nan

for col in missing_2024:
    df_2024[col] = np.nan

# Reorder columns to match, important for concat
df_2023 = df_2023[df_2024.columns]

# Concatenate
df_all = pd.concat([df_2023, df_2024], ignore_index=True)

print(f"Merged shape: {df_all.shape}")


We combined the 2023 and 2024 survey responses into one large dataset. To make this possible, we added any missing questions from one year into the other using empty values (NaNs).

## **Data Preprocessing**

## **Exploratory Data Analysis**

In [ ]:


# List of categorical features to analyze
categorical_features = ['Country', 'Employment', 'RemoteWork', 'OrgSize', 'ICorPM', 
                        'DevType', 'EdLevel', 'Industry', 'PurchaseInfluence',
                        'LanguageHaveWorkedWith', 'PlatformHaveWorkedWith', 
                        'ToolsTechHaveWorkedWith', 'SurveyYear']

# Loop through each feature and plot the top categories
for feature in categorical_features:
    plt.figure(figsize=(12,5))

    # Filter out 'Missing' values first
    df_filtered = df_all[df_all[feature] != 'Missing']

    # Get top categories by frequency
    top_categories = df_filtered[feature].value_counts().nlargest(10).index

    # Filter to top categories
    filtered_df = df_filtered[df_filtered[feature].isin(top_categories)]

    # Compute mean salary per category, sorted by salary
    sorted_order = (
        filtered_df.groupby(feature)['ConvertedCompYearly']
        .mean()
        .sort_values(ascending=False)
        .index
    )

    # Create barplot
    sns.barplot(
        data=filtered_df,
        x=feature,
        y='ConvertedCompYearly',
        estimator=np.mean,
        ci=None,
        order=sorted_order,
        palette='viridis'
    )

    plt.title(f'Average Salary by {feature}')
    plt.xticks(rotation=45)
    plt.ylabel('Average Salary (USD)')
    plt.tight_layout()
    plt.show()


In [ ]:
# Clean data
clean_salary = df_all['ConvertedCompYearly'].dropna()
clean_salary = clean_salary[clean_salary > 1000]  # Remove very low outliers
clean_salary = clean_salary[clean_salary < 300000]  # Cap extreme values

# Plot
plt.figure(figsize=(10,5))
sns.histplot(clean_salary, bins=50, kde=True, color='purple')
plt.axvline(clean_salary.median(), color='red', linestyle='--', label=f"Median: ${int(clean_salary.median()):,}")
plt.title("Distribution of Developer Salaries (USD)")
plt.xlabel("Salary (USD)")
plt.ylabel("Number of Developers")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


### ✍️ Interpretation  
Salaries tend to cluster between $30,000 and $100,000 per year, with a few very high earners pulling the average upward. This is known as a right-skewed distribution, most developers earn within a moderate range, while a few outliers earn significantly more.   
**Consider applying log transformation during modeling.**

In [ ]:
# Define relevant features for salary prediction (features selected using relevance and domain intuition)
selected_features = [
    'Country', 'Employment', 'RemoteWork', 'OrgSize', 'ICorPM',
    'YearsCodePro', 'DevType', 'EdLevel', 'Industry', 
    'PurchaseInfluence', 'LanguageHaveWorkedWith', 'PlatformHaveWorkedWith',
    'ToolsTechHaveWorkedWith', 'ConvertedCompYearly', 'SurveyYear', 'AIThreat'
]

# Subset and copy data
df = df_all[selected_features].copy()

# Drop missing or unrealistic target values
df = df[df['ConvertedCompYearly'].notna()]
df = df[df['ConvertedCompYearly'] > 1000]  # Filter extremely low salaries

# Handle missing categorical values
categorical_cols = df.select_dtypes(include='object').columns
df[categorical_cols] = df[categorical_cols].fillna('Missing')

# Clean numeric features
df['YearsCodePro'] = pd.to_numeric(df['YearsCodePro'], errors='coerce')
df['YearsCodePro'] = df['YearsCodePro'].fillna(df['YearsCodePro'].median())

print(f"✅ Final Cleaned Data Shape: {df.shape}")


In [ ]:
## Salary by years of professional experience
# Filter: Remove missing or unrealistic salary/experience
salary_exp_df = df[
    df['ConvertedCompYearly'].notna() & 
    df['YearsCodePro'].notna()
].copy()

# Bin YearsCodePro for clarity
salary_exp_df['ExperienceBin'] = pd.cut(
    salary_exp_df['YearsCodePro'],
    bins=[0, 2, 5, 10, 15, 20, 30, 50],
    labels=['0–2', '3–5', '6–10', '11–15', '16–20', '21–30', '30+']
)

# Plot
plt.figure(figsize=(10,6))
sns.boxplot(data=salary_exp_df, x='ExperienceBin', y='ConvertedCompYearly', showfliers=False)
plt.title('Salary by Years of Professional Experience')
plt.xlabel('Years of Coding Experience (Binned)')
plt.ylabel('Annual Salary (USD)')
plt.ylim(0, salary_exp_df['ConvertedCompYearly'].quantile(0.95))  # Remove extreme outliers
plt.tight_layout()
plt.show()


The graph shows a clear positive correlation between years of professional experience and annual salary. 
As experience increases, salaries tend to rise steadily.  
Diminishing returns observed after 20+ years.



In [ ]:
numeric_features = df.select_dtypes(include=[np.number])
corr_matrix = numeric_features.corr()

plt.figure(figsize=(8,4))
sns.heatmap(
    corr_matrix[['ConvertedCompYearly']].sort_values(by='ConvertedCompYearly', ascending=False),
    annot=True, cmap='viridis', vmin=-1, vmax=1
)
plt.title("Correlation of Numerical Features with Salary")
plt.tight_layout()
plt.show()


In [ ]:
## Salary by ICorPM (Individual Contributor vs People Manager)
# Filter to remove missing or "Missing" values
ic_pm_df = df[
    df['ConvertedCompYearly'].notna() &
    df['ICorPM'].notna() &
    (df['ICorPM'] != 'Missing')
].copy()

# Plot salary distribution by ICorPM
plt.figure(figsize=(8,5))
sns.boxplot(
    data=ic_pm_df,
    x='ICorPM',
    y='ConvertedCompYearly',
    showfliers=False  # hides extreme outliers
)

plt.title('Salary by Role Type: Individual Contributor vs People Manager')
plt.xlabel('Role Type')
plt.ylabel('Annual Salary (USD)')
plt.ylim(0, ic_pm_df['ConvertedCompYearly'].quantile(0.95))  # limit y-axis to 95th percentile
plt.tight_layout()
plt.show()


The average annual salary for a People Manager role is significantly higher than the average salary for an Individual Contributor role. The graph shows a clear gap between the two.

In [ ]:
# Salary distribution by country (Top 10)
# Get top 10 countries by frequency
top_countries = df['Country'].value_counts().head(10).index

plt.figure(figsize=(14,6))
sns.boxplot(
    data=df[df['Country'].isin(top_countries)],
    x='Country',
    y='ConvertedCompYearly'
)

plt.xticks(rotation=30, ha='right')
plt.ylim(0, 300000) 
plt.title("Salary Distribution by Country (Top 10)")
plt.xlabel("Country")
plt.ylabel("Yearly Compensation (USD)")
plt.tight_layout()
plt.show()




The graph illustrates significant differences in the salary ranges between the top 10 countries. Some countries like the United States and Canada have a much wider salary distribution, while others like Poland and India have a more compressed range.

In [ ]:
# Columns with multiple selections
tech_columns = ['LanguageHaveWorkedWith', 'PlatformHaveWorkedWith', 'ToolsTechHaveWorkedWith']

for col in tech_columns:
    # Filter out missing entries
    tech_df = df[df[col].notna()].copy()
    tech_df = tech_df[tech_df[col] != 'Missing']

    # Split and explode
    tech_df[col] = tech_df[col].str.split(';')
    tech_df = tech_df.explode(col)
    tech_df[col] = tech_df[col].str.strip()

    # Remove 'Missing' after explode just in case
    tech_df = tech_df[tech_df[col] != 'Missing']

    # Group and compute mean salary + count
    salary_by_tech = (
        tech_df.groupby(col)['ConvertedCompYearly']
        .agg(mean_salary='mean', count='count')
        .sort_values(by='count', ascending=False)
        .head(10)
        .sort_values(by='mean_salary', ascending=False)  # sort top 10 by salary
        .reset_index()
    )

    # Plot
    plt.figure(figsize=(12, 5))
    sns.barplot(data=salary_by_tech, x=col, y='mean_salary', palette='viridis')
    plt.title(f"Average Salary by {col.replace('HaveWorkedWith', '')} (Top 10 Most Used)")
    plt.ylabel("Mean Salary (USD)")
    plt.xlabel(col.replace('HaveWorkedWith', ''))
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()


### AIThreat to developer roles
Although we did not use this in our prediction model, we found it useful to visualise how Artificial Intelligence (AI) affects the different developer roles in the market and if any of them felt particularly threatened by the rise and sophistication of AI.

In [ ]:
# Filter clean data
valid_ai_responses = ["Yes", "No", "I'm not sure"]
df_ai = df[df['AIThreat'].isin(valid_ai_responses)].copy()

# Prepare DevType column
df_ai = df_ai.dropna(subset=['DevType'])
df_ai['DevType'] = df_ai['DevType'].str.split(';')
df_ai = df_ai.explode('DevType')
df_ai['DevType'] = df_ai['DevType'].str.strip()

# Normalize role threat responses
# Total count of each role overall
role_totals = df_ai['DevType'].value_counts()

# Function to calculate top 10 roles by percentage for a given threat category
def get_top_roles(threat_label, color):
    threat_df = df_ai[df_ai['AIThreat'] == threat_label]
    role_counts = threat_df['DevType'].value_counts()
    role_percent = (role_counts / role_totals * 100).dropna()
    top_roles = role_percent.nlargest(10)
    
    # Plot
    plt.figure(figsize=(10, 5))
    sns.barplot(
        x=top_roles.values,
        y=top_roles.index,
        palette=color
    )
    plt.title(f"Top 10 Roles Most Likely to Respond '{threat_label}' (by %)")
    plt.xlabel("Percentage of Role Respondents")
    plt.ylabel("Developer Role")
    plt.xlim(0, 100)
    plt.tight_layout()
    plt.show()
    
    return top_roles

# Step 4: Run for each category
top_yes = get_top_roles("Yes", "Reds")
top_no = get_top_roles("No", "Greens")
top_unsure = get_top_roles("I'm not sure", "YlOrBr")



From the results we can see a small number of professionals feel threatened by the rise of AI to their roles. In contrast we see a large number of developers confident that AI will not be a threat to their roles which is encouraging to note. We also had a couple of developers that were unsure of their fate.

In [ ]:
# Sub-Saharan Africa Salary Distribution Boxplot
ssa_countries = [
    'Angola', 'Benin', 'Botswana', 'Burkina Faso', 'Burundi', 'Cape Verde', 'Cameroon',
    'Central African Republic', 'Chad', 'Comoros', 'Democratic Republic of the Congo', 'Congo', 'Congo, Republic of the...',
    'Côte d’Ivoire', "Côte d'Ivoire", 'Djibouti', 'Equatorial Guinea', 'Eritrea', 'Eswatini', 'Ethiopia', 
    'Gabon', 'Gambia', 'Ghana', 'Guinea', 'Guinea-Bissau', 'Kenya', 'Lesotho', 'Liberia',
    'Madagascar', 'Malawi', 'Mali', 'Mauritania', 'Mauritius', 'Mozambique', 'Namibia',
    'Niger', 'Nigeria', 'Rwanda', 'São Tomé and Príncipe', 'Senegal', 'Seychelles', 
    'Sierra Leone', 'Somalia', 'South Africa', 'South Sudan', 'Sudan', 'Swaziland', 'Tanzania', 
    'Togo', 'Uganda', 'United Republic of Tanzania', 'Zambia', 'Zimbabwe'
]
# ✅ Filter SSA responses
ssa_freq = df[df['Country'].isin(ssa_countries)]['Country'].value_counts()

# ✅ Display sorted frequencies
print("📊 Response Counts by Sub-Saharan Country:\n")
print(ssa_freq)

In [ ]:
# 2️⃣ Filter to SSA countries with valid salary data
ssa_df = df[
    (df['Country'].isin(ssa_countries)) &
    (df['ConvertedCompYearly'].notna())
].copy()

# Optional: Remove outliers beyond the 95th percentile
ssa_df = ssa_df[
    ssa_df['ConvertedCompYearly'] < ssa_df['ConvertedCompYearly'].quantile(0.95)
]

# 3️⃣ Get top 10 SSA countries by response count
top_10_ssa_countries = ssa_df['Country'].value_counts().nlargest(10).index

# 4️⃣ Filter dataset to top 10 SSA countries
top_10_ssa_df = ssa_df[ssa_df['Country'].isin(top_10_ssa_countries)]

# 5️⃣ Compute average salary and sort
mean_salaries = (
    top_10_ssa_df.groupby('Country')['ConvertedCompYearly']
    .mean()
    .sort_values(ascending=False)
)

# 6️⃣ Plot
plt.figure(figsize=(10, 6))
sns.barplot(
    x=mean_salaries.values,
    y=mean_salaries.index,
    palette='viridis'
)

# ✅ Annotate values
for i, value in enumerate(mean_salaries.values):
    plt.text(value + 500, i, f"${int(value):,}", va='center')
    
plt.title('🌍 Average Developer Salary by SSA Country (Top 10)', fontsize=14)
plt.xlabel('Average Salary (USD)')
plt.ylabel('Country')
plt.tight_layout()
plt.show()

As per this analysis, South Africa, Zimbabwe and Kenya emerged as the countries in Kenya with the heighest average developer salary based.

In [ ]:
# Reuse previous top_10_ssa_df (already filtered)
# Ensure outliers are removed for better visualization
filtered_box_df = top_10_ssa_df[
    top_10_ssa_df['ConvertedCompYearly'] < top_10_ssa_df['ConvertedCompYearly'].quantile(0.95)
]

# Sort countries by median salary
ordered_countries = (
    filtered_box_df.groupby('Country')['ConvertedCompYearly']
    .median()
    .sort_values(ascending=False)
    .index
)

# Plot the boxplot
plt.figure(figsize=(12, 6))
sns.boxplot(
    data=filtered_box_df,
    x='Country',
    y='ConvertedCompYearly',
    order=ordered_countries,
    palette='viridis',
    showfliers=False  # Removes extreme outliers
)

plt.title("💼 Salary Distribution by Country (Top 10 SSA)", fontsize=14)
plt.xlabel("Country")
plt.ylabel("Annual Salary (USD)")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


South Africa has the highest annual compensation levels, with the top end reaching around $80,000. The rest of the countries predominantly fall between the $10,000 - $30,000 range

In [ ]:
## Salary Distribution by Developer Role (DevType)
# Filter: keep rows with salary and DevType info
role_df = df[
    df['ConvertedCompYearly'].notna() &
    df['DevType'].notna() &
    (df['DevType'] != 'Missing')
].copy()

# Limit to top 10 most common roles to make the plot readable
top_roles = role_df['DevType'].value_counts().nlargest(10).index
role_df = role_df[role_df['DevType'].isin(top_roles)]

# Plot
plt.figure(figsize=(12, 6))
sns.boxplot(
    data=role_df,
    x='DevType',
    y='ConvertedCompYearly',
    showfliers=False
)

plt.title('Salary Distribution by Developer Role')
plt.xlabel('Developer Role')
plt.ylabel('Annual Salary (USD)')
plt.xticks(rotation=45, ha='right')
plt.ylim(0, role_df['ConvertedCompYearly'].quantile(0.95))  # Clip top 5% to reduce skew
plt.tight_layout()
plt.show()



The roles with the highest salary ranges are Engineering Manager, Data scientist/ machine learning specialist and DevOps Specialist. These roles seem to have the potential for the highest annual compensation, reaching up to around $175,000 or more.

---

## **🛠️ Feature Engineering**  
In this section, we'll create new features, clean up the data further, and prepare it for our model. This ensures our model understands the data better and can predict salaries more accurately. We aim to:    

        -   Makes the data easier for the model to understand  
        -   Highlights important patterns (e.g., experience level or tech skills)  
        -   Helps address issues like missing or skewed data


### 🛠️ Feature Engineering: Creating New Features  
We're creating new data points (features) to help our model predict salaries better. For example:,
- Counting how many programming languages, tools and platforms a developer knows,
- Grouping countries into regions (e.g., sub-saharan Africa, North America),
- Simplifying education levels into categories like 'Bachelor’s' or 'No degree' 

This makes it easier for our model to find patterns, like whether knowing more programming languages leads to higher pay.  

#### 1. Create a Working Copy of the Dataset and Select Relevant Columns

In [ ]:
df = df_all.copy()

# Drop rows with missing salary
df = df[df['ConvertedCompYearly'].notna()]

# Select relevant columns
selected_columns = [
    'ConvertedCompYearly', 'Country', 'EdLevel', 'YearsCodePro',
    'Employment', 'OrgSize', 'RemoteWork', 'LanguageHaveWorkedWith',
    'DevType', 'SurveyYear', 'ICorPM', 'Industry', 
    'PurchaseInfluence', 'PlatformHaveWorkedWith', 'ToolsTechHaveWorkedWith'
]
df = df[selected_columns]
df

#### 2. Create a New Feature: Number of Programming Languages, Platforms and Tools Known

In [ ]:
df['NumLanguages'] = df['LanguageHaveWorkedWith'].apply(lambda x: len(str(x).split(';')) if pd.notna(x) else 0)
df['NumPlatforms'] = df['PlatformHaveWorkedWith'].apply(lambda x: len(str(x).split(';')) if pd.notna(x) else 0)
df['NumTools'] = df['ToolsTechHaveWorkedWith'].apply(lambda x: len(str(x).split(';')) if pd.notna(x) else 0)

#### 3. Simplify Education Levels

In [ ]:
df['EdLevel'].unique()

In [ ]:
# Simplify education
def simplify_edlevel(edlevel):
    if pd.isna(edlevel):
        return 'Unknown'
    edlevel = str(edlevel)
    if 'Bachelor' in edlevel:
        return 'Bachelor’s'
    elif 'Master' in edlevel:
        return 'Master’s'
    elif 'Professional' in edlevel or 'Ph.D' in edlevel or 'Ed.D' in edlevel or 'MD' in edlevel or 'JD' in edlevel:
        return 'Doctorate/Professional'
    elif 'Associate' in edlevel:
        return 'Associate'
    elif 'Secondary school' in edlevel or 'Some college' in edlevel:
        return 'Some College/Secondary'
    elif 'Primary' in edlevel:
        return 'Primary'
    elif 'Something else' in edlevel:
        return 'Other'
    else:
        return 'Unknown'

df['EdLevel_Simplified'] = df['EdLevel'].apply(simplify_edlevel) 
df['EdLevel'].apply(simplify_edlevel)


#### 4. Grouping countires into regions

In [ ]:
# Count number of respondents per country
df['Country'].value_counts(ascending= False).reset_index().head(20)



In [ ]:
# Keep your selected top 10 SSA countries separate
african_countries = ssa_countries

# Define a function to assign regions
def get_region(country):
    if pd.isna(country): 
        return 'Unknown'
    elif country == 'Kenya': 
        return 'Kenya'
    elif country in african_countries: 
        return 'Sub Saharan Africa'
    elif country in ['United States of America', 'Canada']: 
        return 'North America'
    elif country in ['United Kingdom of Great Britain and Northern Ireland', 'Germany', 'France', 'Italy', 'Netherlands', 'Sweden', 'Switzerland', 'Austria', 'Belgium', 'Portugal',
                    'Spain', 'Greece', 'Norway', 'Iceland', 'Denmark', 'Finland', 'Ireland', 'Estonia', 'Lithuania', 'Latvia', 'Luxembourg', 'Monaco']: 
        return 'Western & Northern Europe'
    elif country in ['Argentina', 'Bolivia', 'Brazil', 'Chile', 'Colombia', 'Costa Rica', 'Cuba', 'Dominican Republic', 'Ecuador', 
                    'El Salvador', 'Guatemala', 'Honduras', 'Mexico', 'Nicaragua', 'Panama', 'Paraguay', 'Peru', 'Uruguay', 'Venezuela', 'Venezuela, Bolivarian Republic of...']: 
        return 'Latin America'
    elif country in ['India', 'Pakistan', 'Bangladesh', 'Sri Lanka', 'Indonesia', 'Nepal', 'Vietnam', 'Philippines', 'Malaysia', 'Thailand', 
                    'Bhutan', 'Maldives', 'Brunei', 'Cambodia', 'Laos', 'Myanmar','Viet Nam']:
        return 'South & Southeast Asia'
    elif country in ['China', 'Japan', 'South Korea', 'Taiwan', 'Hong Kong', 'Hong Kong (S.A.R.)', 'Singapore', 'Macau', 'Mongolia', 'North Korea', 'Republic of Korea', "Democratic People's Republic of Korea"]: 
        return 'East Asia'
    elif country in ['Armenia', 'Albenia','Armenia', 'Montenegro', 'Poland', 'Ukraine', 'Romania', 'Russian Federation', 'Serbia', 'Czech Republic', 'Slovakia', 'Hungary', 'Moldova', 'Republic of Moldova', 'Belarus', 'Bulgaria', 'Kosovo', 'Slovenia',
                    'Croatia', 'Bosnia and Herzegovina', 'Kazakhstan', 'Uzbekistan', 'Albania', 'Azerbaijan', 'Kyrgyzstan', 'Afghanistan']:
        return 'Eastern Europe & Central Asia'
    elif country in ['Australia', 'New Zealand', 'Fiji']: 
        return 'Oceania'
    elif country in ['Antigua and Barbuda', 'Bahamas', 'Barbados', 'Cuba', 'Dominica',' Dominican Republic',' Grenada', 'Haiti', 'Jamaica', 
                    'Saint Kitts and Nevis', 'Saint Lucia',' Saint Vincent',' The Grenadines', 'Trinidad and Tobago', 'Anguilla', 'Aruba', 
                    'British Virgin Islands', 'Cayman Islands', 'Puerto Rico']:
        return 'Caribbean'
    elif country in ['Algeria', 'Egypt', 'Libya', 'Morocco', 'Tunisia', 'Mauritania',
                 'Bahrain', 'Iran, Islamic Republic of...', 'Iraq', 'Israel', 'Jordan', 'Kuwait', 'Lebanon', 'Oman', 'Cyprus',
                 'Turkey', 'Georgia', 'Malta', 'Palestine', 'Qatar', 'Saudi Arabia', 'Syria', 'Syrian Arab Republic', 'United Arab Emirates', 'Yemen']: 
         return 'MENA'
    else: return 'Other'

# Apply the region mapping
df['Region'] = df['Country'].apply(get_region)


In [ ]:
df[df['Region'] == 'Other']['Country'].value_counts()

#### 5. Grouping Employment

In [ ]:
# Engineer Employment
def simplify_employment(employment):
    if pd.isna(employment) or 'I prefer not to say' in str(employment):
        return 'Unknown'
    employment_list = str(employment).split(';')
    priority_order = [
        'Employed, full-time',
        'Independent contractor, freelancer, or self-employed',
        'Employed, part-time',
        'Student, full-time',
        'Student, part-time',
        'Not employed, but looking for work',
        'Not employed, and not looking for work',
        'Retired'
    ]
    for status in priority_order:
        if status in employment_list:
            if status == 'Employed, full-time': return 'Full-time'
            elif status == 'Independent contractor, freelancer, or self-employed': return 'Freelance'
            elif status == 'Employed, part-time': return 'Part-time'
            elif status in ['Student, full-time', 'Student, part-time']: return 'Student'
            elif status == 'Not employed, but looking for work': return 'Unemployed'
            elif status == 'Not employed, and not looking for work': return 'Unemployed'
            elif status == 'Retired': return 'Retired'
    return 'Unknown'
df['Employment_Simplified'] = df['Employment'].apply(simplify_employment)
df['Employment'].apply(simplify_employment)

#### 6. Ensuring the `YearsCodePro` is numeric and sorting them into bins

In [ ]:
# Convert and Clean YearsCodePro
df['YearsCodePro'] = pd.to_numeric(df['YearsCodePro'], errors='coerce')
df['YearsCodePro'].fillna(df['YearsCodePro'].median(), inplace=True)

# Binned experience
def bin_experience(years):
    if pd.isna(years): return 'Unknown'
    years = float(years)
    if years <= 3: return 'Beginner'
    elif years <= 7: return 'Intermediate'
    elif years <= 12: return 'Advanced'
    else: return 'Expert'

df['ExperienceLevel'] = df['YearsCodePro'].apply(bin_experience)

#### 7. Grouping `OrgSize` into `small`, `medium` and `large`

In [ ]:
def simplify_orgsize(size):
    if pd.isna(size): return 'Unknown'
    if 'fewer than 10' in size or '10 to 19' in size: return 'Small'
    elif '20 to 99' in size or '100 to 499' in size: return 'Medium'
    elif '500 to 999' in size or '1,000 or more' in size: return 'Large'
    else: return 'Unknown'
df['OrgSize_Simplified'] = df['OrgSize'].apply(simplify_orgsize)

#### 8. Binary classification for whether the role is managerial

In [ ]:
df['ICorPM'] = df['ICorPM'].apply(lambda x: 1 if 'Manager' in str(x) else 0)

#### 9. Grouping Industries into simplified buckets

In [ ]:
# Engineer Industry
def simplify_industry(industry):
    if pd.isna(industry):
        return 'Unknown'
    industry = str(industry).replace('Other:', 'Other')  # Normalize "Other:"
    if any(keyword in industry for keyword in ['Information Services', 'IT', 'Software Development', 'Computer Systems Design', 'Internet, Telecomm']):
        return 'Tech'
    elif any(keyword in industry for keyword in ['Financial', 'Banking', 'Fintech', 'Insurance']):
        return 'Finance'
    elif any(keyword in industry for keyword in ['Manufacturing', 'Transportation', 'Supply Chain', 'Wholesale', 'Oil', 'Energy']):
        return 'Manufacturing/Supply Chain'
    elif any(keyword in industry for keyword in ['Retail', 'Consumer Services', 'Higher Education', 'Legal', 'Advertising', 'Media']):
        return 'Services'
    elif 'Healthcare' in industry:
        return 'Healthcare'
    elif 'Government' in industry:
        return 'Government'
    elif 'Other' in industry:
        return 'Other'
    return 'Unknown'

df['Industry_Simplified'] = df['Industry'].apply(simplify_industry)
df['Industry'].apply(simplify_industry)

#### 10. Simplifying the `RemoteWork` category

In [ ]:
# Simplify Remote Work Categories
def simplify_remote(remote):
    if pd.isna(remote):
        return 'Unknown'
    elif 'Remote' in remote:
        return 'Remote'
    elif 'Hybrid' in remote:
        return 'Hybrid'
    else:
        return 'In-person'

df['RemoteWork_Simplified'] = df['RemoteWork'].apply(simplify_remote)

#### 11. Log transformation on target variable for easier modelling

In [ ]:
# Log Transform Salary 
df['Log_ConvertedCompYearly'] = np.log1p(df['ConvertedCompYearly'])

#### 12. Creating a new column `RegionIncomeLevel`

In [ ]:
# Region-based income level (World Bank style)
income_map = {
    'Kenya': 'Lower-Middle',
    'Sub Saharan Africa': 'Lower-Middle',
    'North America': 'High',
    'Western & Northern Europe': 'High',
    'Eastern Europe': 'Upper-Middle',
    'South & Southeast Asia': 'Lower-Middle',
    'East Asia': 'High',
    'MENA': 'Upper-Middle',
    'Latin America': 'Upper-Middle',
    'Other': 'Unknown'
}
df['RegionIncomeLevel'] = df['Region'].map(income_map)


#### 13. Creating `Region_Remote` column
You are creating a new feature called Region_Remote, which combines the values of:

Region (e.g., "Kenya", "North America")
RemoteWork_Simplified (e.g., "Remote", "Hybrid", "On-site")
🎯 Purpose:
You're helping the model answer questions like:

💭 "Does being remote in North America affect salary differently than being remote in Kenya?"
Because:

A remote developer in North America might earn more due to access to high-paying markets.
A remote developer in Kenya might still earn less due to location-based bias or fewer global clients.
By combining Region and RemoteWork_Simplified, the model can learn these nuanced patterns.

In [ ]:

# Interaction term: Region × Remote Work
df['Region_Remote'] = df['Region'] + '_' + df['RemoteWork_Simplified']

In [ ]:
print(f"Shape after feature engineering: {df.shape}")
df.head()

🔍 What We Did  

- Number of Languages: Counted how many programming languages, tools and platforms each developer knows to capture skill diversity.
- Simplified Education: Grouped education levels into broader categories (e.g., Bachelor’s, Master’s) to make them easier to analyze.
- Region Grouping: Organized countries into regions, keeping Kenya separate for focus, while grouping others like North America or Europe.
- Years of Experience: Converted years of professional coding experience to numbers and filled in missing values with the median.
- Remote Work: Simplified remote work status into Remote, Hybrid, or In-person.
- Salary Transformation: Applied a logarithmic transformation to salaries to reduce the impact of extreme values (e.g., very high salaries).

NOTE: Use the number of tools, platforms, and languages (counts) as the primary approach for 'PlatformHaveWorkedWith', 'ToolsTechHaveWorkedWith', and 'LanguageHaveWorkedWith'. Reason being:

- Efficiency with Sparse Data: Counts (NumPlatforms, NumTools, NumLanguages) reduce dimensionality, mitigating overfitting risks in SSA/Kenya.
- General Salary Proxy: Skill diversity correlates with experience and marketability, a reliable predictor across regions, including SSA.
- Flexibility: You can validate this with feature importance (e.g., SHAP) and switch to specific types if counts show low impact.  
**Hybrid Option: If SHAP later indicates high importance for specific skills (e.g., "Python" or "AWS"), we can engineer top-N categories (e.g., top 5 languages) as additional binary features. Starting with counts to establish a baseline.**

These steps help our model focus on the most important patterns in the data.

---

## **⚖️ Handling Imbalanced Data**

Our dataset has a lot more responses from some countries (like the US) than others (like Kenya). This can throw off the final result.

To fix this, we use techniques to balance the data, so our model pays attention to all regions, especially smaller ones like Kenya.

Since we’re predicting a number (salary), we can’t directly use oversampling techniques like SMOTE.
Instead, we ensure balance by using stratified sampling and possibly sample weighting during modeling.

In [ ]:
# Check distribution of regions
print("Region distribution:\n", df['Region'].value_counts())

# Limit dataset to the top 10 developer roles to reduce noise
top_roles = df['DevType'].value_counts().nlargest(10).index
df = df[df['DevType'].isin(top_roles)]

print(f"Shape after filtering: {df.shape}")

✅ What We Did
- Checked how many responses come from each region to understand imbalance.
- Focused on the top 10 developer roles to keep the data manageable and relevant.
This helps ensure our model isn’t overwhelmed by less relevant data and can focus on key patterns.

---

## **Splitting the Data**
 
To make sure our model can predict salaries accurately for new data, we split our dataset into two parts:  

- Training set (80%): Used to teach the model how to predict salaries.  
- Test set (20%): Used to check how well the model performs on new data. 

This is like practicing for a test with most of the material but saving a few questions to see how you do without help.

In [ ]:
from sklearn.model_selection import train_test_split

# Define features and target
features = [ 'EdLevel_Simplified', 'Region', 'ExperienceLevel', 'RegionIncomeLevel', 'Region_Remote',
    'RemoteWork_Simplified', 'DevType', 'NumLanguages',
    'Employment_Simplified', 'OrgSize_Simplified', 'ICorPM', 'Industry_Simplified', 'SurveyYear', 
    'NumTools', 'NumPlatforms'
]
target = 'Log_ConvertedCompYearly'

# Split into features and target
X = df[features]
y = df[target]

# Stratified split by region
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=X['Region']
)

# Oversample Kenya in the training set
kenya_data = X_train[X_train['Region'] == 'Kenya']
kenya_labels = y_train[kenya_data.index]

X_train_bal = pd.concat([X_train, kenya_data]*3, ignore_index=True)
y_train_bal = pd.concat([y_train, kenya_labels]*3, ignore_index=True)

print(f"Training set shape: {X_train.shape}, Test set shape: {X_test.shape}")


✅ What We Did
- Selected key features like education, region, and number of languages to predict salaries.
- Used the log-transformed salary as our target to handle skewed data.
- Split the dataset into 80% training and 20% testing while preserving regional balance.  

This prepares our data for modeling in a way that generalizes well to new data.

📊 Visualizing Key Features

To understand our new features, let’s visualize how log salary varies by region and number of programming languages.

In [ ]:
# Boxplot: Log Salary by Region
plt.figure(figsize=(12, 6))
sns.boxplot(data=df, x='Region', y='Log_ConvertedCompYearly', showfliers=False)
plt.title('Log Salary Distribution by Region')
plt.xlabel('Region')
plt.ylabel('Log Annual Salary (USD)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

# Scatterplot: Log Salary vs Number of Programming Languages
plt.figure(figsize=(12, 6))
sns.scatterplot(data=df, x='NumLanguages', y='Log_ConvertedCompYearly', alpha=0.5)
plt.title('Log Salary vs Number of Programming Languages')
plt.xlabel('Number of Programming Languages')
plt.ylabel('Log Annual Salary (USD)')
plt.tight_layout()
plt.show()


### 📊 Graph 1: Log Salary Distribution by Region

This **boxplot** compares the **log-transformed annual salary** of developers across different **global regions**.

#### 🔍 What It Shows:
- Each box represents the salary distribution within a region.

#### ✅ Key Insights:
- **North America** has one of the highest median salaries and a narrower range, suggesting consistently high developer pay.
- **Sub-Saharan Africa** and **North Africa** show lower median salaries with a wider range, indicating more salary variability (Kenya included).
- **Western Europe** and **Oceania** also rank high in terms of median salaries.
- The log scale helps normalize extreme salary values to make patterns clearer across regions.

#### 💡 Takeaway:
Where a developer lives can significantly influence their salary. Developers in North America and Western Europe tend to earn more than those in regions like Sub-Saharan Africa or South Asia.

---

### 📊 Graph 2: Log Salary vs Number of Programming Languages

This **scatterplot** shows the relationship between how many **programming languages** a developer knows and their **log-transformed salary**.

#### 🔍 What It Shows:
- Each point represents a developer.
- The x-axis is the number of programming languages they reported working with.
- The y-axis is their log-transformed annual salary.

#### ✅ Key Insights:
- There's a **slight upward trend** — developers who know more languages tend to earn more.
- Most developers fall within the **1–10 language range**, with highly varied salaries.
- Beyond ~10 languages, the correlation becomes weaker and more scattered.
- Some high-salary outliers exist across all levels, indicating other factors (e.g., region, job title, experience) also matter.

#### 💡 Takeaway:
Knowing more programming languages can **improve earning potential**, but it’s **not the only factor**. Salary outcomes also depend on experience, region, remote work status, job role, and other attributes.

---

📝 These two visualizations help us understand how **location** and **skill diversity** relate to developer salaries. They're key for guiding how we structure our prediction model and feature selection.


## 🛠️ **Modeling Pipeline**

### What is a Modeling Pipeline?
A modeling pipeline is like a recipe for making predictions. It combines steps to prepare the data (like turning categories into numbers) and then uses a math tool (model) to predict salaries. We’ll try a few models to see which one works best:
- **Baseline (Linear Regression)**: A simple model that assumes salary changes in a straight-line way with things like experience or education.
- **Random Forest**: A smarter model that looks at patterns in the data by combining many small predictions.
- **XGBoost**: A powerful model that’s great for complex data like ours.

We’ll use a pipeline to make sure our data is prepared the same way for each model, avoiding mistakes.

### Why It Matters
- Ensures our predictions are consistent and fair.
- Lets us compare different models to find the best one.
- Saves time by automating data preparation and modeling.

---


In [ ]:

# Define numerical and categorical features
numerical_features = ['NumLanguages', 'NumTools', 'NumPlatforms']
categorical_features = ['EdLevel_Simplified', 'Region', 'RemoteWork_Simplified', 'DevType', 'SurveyYear', 'Employment_Simplified', 'OrgSize_Simplified', 'ICorPM', 'Industry_Simplified', 
    'ExperienceLevel', 'RegionIncomeLevel', 'Region_Remote']

# Create preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ]
)

In [ ]:
# Function to evaluate models
def evaluate_model(model, X_train, X_test, y_train, y_test, model_name):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    print(f"{model_name} RMSE: {rmse:.4f}")
    print(f"{model_name} R2 Score: {r2:.4f}")
    
    kenya_test = X_test[X_test['Region'] == 'Kenya']
    if not kenya_test.empty:
        y_pred_kenya = model.predict(kenya_test)
        rmse_kenya = np.sqrt(mean_squared_error(y_test[kenya_test.index], y_pred_kenya))
        print(f"{model_name} Kenya RMSE: {rmse_kenya:.4f}")
    else:
        print(f"{model_name} Kenya RMSE: No Kenya data in test set")
    return model


In [ ]:
# Baseline: Linear Regression
lr_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])
lr_model = evaluate_model(lr_pipeline, X_train, X_test, y_train, y_test, "Linear Regression")


In [ ]:
# Random Forest
rf_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])
rf_model = evaluate_model(rf_pipeline, X_train, X_test, y_train, y_test, "Random Forest")


In [ ]:
# XGBoost
xgb_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(n_estimators=100, random_state=42))
])
xgb_model = evaluate_model(xgb_pipeline, X_train, X_test, y_train, y_test, "XGBoost")


What We Did in Modeling  
- Set Up a Pipeline: Created a system to prepare data (scaling numbers, encoding categories) and run models consistently.
- Tried Three Models:
    - Linear Regression: A simple model to get a starting point.
    - Random Forest: A model that combines many predictions for better accuracy.
    - XGBoost: A powerful model for complex patterns.  
- Evaluated for Kenya: Checked how well each model predicts salaries for Kenyan developers.

What’s a Good Score?
- We’re measuring error using RMSE (Root Mean Squared Error) on the adjusted (log) salaries.
- A good RMSE is 0.35–0.45, meaning predictions are within ±40% of the actual salary.
- We also check the R2 score, which shows how much variation our model explains (closer to 1.0 is better).

## 📊 Evaluation

How We Judge Success
We use RMSE to see how close our predictions are to actual salaries. Since salaries vary widely, we use log salaries to keep errors consistent. We also look at Kenya RMSE to ensure smaller regions are represented fairly.

What the Results Mean
- RMSE of 0.35–0.45: Acceptably close predictions.
- Kenya RMSE: Crucial for localized accuracy.
- R2 Score: Shows how well the model explains salary variation.

In [ ]:
 # Plot actual vs predicted salaries for the best model (XGBoost)
y_pred_xgb = xgb_pipeline.predict(X_test)
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_xgb, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual Log Salary')
plt.ylabel('Predicted Log Salary')
plt.title('Actual vs Predicted Log Salaries (XGBoost)')
plt.tight_layout()
plt.show()

# Kenya-specific plot
kenya_test = X_test[X_test['Region'] == 'Kenya']
if not kenya_test.empty:
    y_pred_kenya = xgb_pipeline.predict(kenya_test)
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test[kenya_test.index], y_pred_kenya, alpha=0.5, color='green')
    plt.plot(
        [y_test[kenya_test.index].min(), y_test[kenya_test.index].max()],
        [y_test[kenya_test.index].min(), y_test[kenya_test.index].max()],
        'r--', lw=2
    )
    plt.xlabel('Actual Log Salary (Kenya)')
    plt.ylabel('Predicted Log Salary (Kenya)')
    plt.title('Actual vs Predicted Log Salaries for Kenya (XGBoost)')
    plt.tight_layout()
    plt.show()
else:
    print("No Kenya data in test set for plotting.")


What These Plots Show
- Actual vs Predicted: Points close to the red line are accurate predictions.
- Kenya Plot: Shows model performance specifically for Kenyan developers.

---
## 🔍 Model Interpretation (Explainability)

Why Explain the Model?
Our prediction tool is a black box. We use SHAP to understand which features (like region or experience) most influence predictions. This makes results trustworthy for decision-making.

What is SHAP?
SHAP (SHapley Additive exPlanations) shows how much each feature contributes to a prediction. For example, does being in Kenya increase or decrease salary predictions?

In [ ]:
# Use SHAP to explain the XGBoost model
X_test_transformed = xgb_pipeline.named_steps['preprocessor'].transform(X_test)

# Get feature names after preprocessing
cat_features_encoded = xgb_pipeline.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(categorical_features)
feature_names = numerical_features + list(cat_features_encoded)

# Create SHAP explainer
explainer = shap.TreeExplainer(xgb_pipeline.named_steps['regressor'])
shap_values = explainer.shap_values(X_test_transformed)

# SHAP summary
plt.figure(figsize=(12, 8))
shap.summary_plot(shap_values, X_test_transformed, feature_names=feature_names, max_display=10)
plt.title('Feature Importance for Salary Predictions (XGBoost)')
plt.tight_layout()
plt.show()

# SHAP for Kenya
kenya_test = X_test[X_test['Region'] == 'Kenya']
if not kenya_test.empty:
    kenya_test_transformed = xgb_pipeline.named_steps['preprocessor'].transform(kenya_test)
    shap_values_kenya = explainer.shap_values(kenya_test_transformed)
    plt.figure(figsize=(12, 8))
    shap.summary_plot(shap_values_kenya, kenya_test_transformed, feature_names=feature_names, max_display=10)
    plt.title('Feature Importance for Salary Predictions in Kenya (XGBoost)')
    plt.tight_layout()
    plt.show()
else:
    print("No Kenya data in test set for SHAP analysis.")


What SHAP Tells Us
- Feature Importance: Red = higher values that increase salary; Blue = values that reduce it.
- Kenya-Specific: Lets us see if the same patterns apply locally.

# LightGBM Regression

In [ ]:
import lightgbm as lgb

In [1]:
# LightGBM Pipeline
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', lgb.LGBMRegressor(
        n_estimators=100,
        learning_rate=0.1,
        num_leaves=31,
        random_state=42
    ))
])

# Train Model
model.fit(X_train, y_train)

# Predict and Evaluate 
y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("Evaluation Metrics:")
print(f"MAE:  {mae:,.2f}")
print(f"RMSE: {rmse:,.2f}")
print(f"R²:   {r2:.4f}")

NameError: name 'Pipeline' is not defined

# Feature Importance 

In [ ]:

# Extract trained model
regressor = model.named_steps['regressor']
feature_names = model.named_steps['preprocessor'].get_feature_names_out()
importances = regressor.feature_importances_

# Show top 15
feat_imp = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
feat_imp.sort_values(by='Importance', ascending=False).head(20).plot(
    kind='barh', x='Feature', y='Importance', title="Top 20 Feature Importances", figsize=(10, 6)
)
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

 **Top 3 Features**

- **`num__NumTools`**  
  → Number of tools a developer uses (e.g., Git, Docker, etc.)  
   Strongest indicator of salary level — possibly because using more tools reflects broader expertise and adaptability.

- **`num__NumLanguages`**  
  → Number of programming languages known  
   Higher number of languages is linked to higher pay.

- **`num__NumPlatforms`**  
  → Number of platforms a developer works on (e.g., web, mobile, desktop)  
   Versatility across platforms increases value.


 **Education Features**

- **`cat__EdLevel_Simplified_Master’s`**  
  → Developer has a Master's degree  
   Having a Master's degree contributes positively to salary, reflecting higher qualifications.

- **`cat__EdLevel_Simplified_Some College/Secondary`**  
  → Developer has some college or secondary education  
   This level of education shows a moderate impact on salary, possibly indicating a starting point in the career.

- **`cat__EdLevel_Simplified_Bachelor’s`**  
  → Developer has a Bachelor’s degree  
   A Bachelor's degree seems to be a key milestone in determining salary, though not as impactful as advanced degrees.


# Ensemble

## we ensemble so as to blend the strengths of the previous models to give us a more balanced and reliable prediction.

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import StackingRegressor, RandomForestRegressor
from sklearn.linear_model import RidgeCV
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

#  Drop rows where the target is missing
df_model = df_all.dropna(subset=['ConvertedCompYearly'])

#  Define features and target
X = df_model.drop(columns=['ConvertedCompYearly'])
y = df_model['ConvertedCompYearly']

#  Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include='object').columns.tolist()
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Define the preprocessor with sparse_output=True to avoid memory explosion
from sklearn.impute import SimpleImputer

# Preprocessing for numerical columns
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical columns
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='Missing')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=True))
])

# Combine both
preprocessor = ColumnTransformer([
    ('num', num_pipeline, numerical_cols),
    ('cat', cat_pipeline, categorical_cols)
])


#  Define base learners
base_learners = [
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('xgb', XGBRegressor(n_estimators=100, random_state=42)),
    ('lgbm', LGBMRegressor(n_estimators=100, random_state=42))
]

# Define the meta learner
meta_learner = RidgeCV()

# Step 7: Assemble the stacking model pipeline
stacking_model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', StackingRegressor(
        estimators=base_learners,
        final_estimator=meta_learner,
        passthrough=True))
])

# Train the model
stacking_model.fit(X_train, y_train)

#  Predict and evaluate
y_pred_stack = stacking_model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred_stack)
rmse = mean_squared_error(y_test, y_pred_stack, squared=False)
r2 = r2_score(y_test, y_pred_stack)

# Step 10: Print evaluation metrics
print("✅ Ensemble Evaluation Metrics:")
print(f"MAE:  {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²:   {r2:.4f}")
